In [1]:
import numpy as np
import pandas as pd

In [2]:
input_file_aggr = '../output/alpha_0.65_corrected_aggr/output_file_1.evaluated.filtered_0.67' # this is the 'output_file_1.evaluated.filtered_0.67' file
input_file_non_aggr = '../output/alpha_0.65_corrected_non_aggr/output_file_1.evaluated.filtered_0.67'
ranking_file_aggr = '../output/alpha_0.65_corrected_aggr/output_file_1.evaluated.filtered_0.67.probabilistic_network.desc'
ranking_file_non_aggr = '../output/alpha_0.65_corrected_non_aggr/output_file_1.evaluated.filtered_0.67.probabilistic_network.desc'
N = 50 # Only a gene that is in the top N of aggresive or non-aggresive (or both) is considered
output_file = 'subnetwork_0.65_corrected_difference_aggr_non_aggr'+str(N)+'.js'

In [3]:
df_aggr = pd.read_csv(input_file_aggr,sep='[',skiprows=0,header=None)

In [4]:
df_non_aggr = pd.read_csv(input_file_non_aggr,sep='[',skiprows=0,header=None)

In [5]:
def ParseDataframe(df):
    df = df.rename(columns={0: 'path_weight', 1: 'genes_path'})
    df[['gene1','gene2','gene3']] = df['genes_path'].str.split(',',expand=True)
    df[['gene3','brol']] = df['gene3'].str.split(']',expand=True)
    df = df.drop(['brol','genes_path'], axis=1)
    df[['path_weight','brol']] = df['path_weight'].str.rsplit('-',1,expand=True)
    df = df.drop(['brol'], axis=1)
    return df

In [6]:
df_aggr = ParseDataframe(df_aggr)
df_aggr

/tmp/ipykernel_10329/683794502.py:6: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df[['path_weight','brol']] = df['path_weight'].str.rsplit('-',1,expand=True)


,path_weight,gene1,gene2,gene3
0,6.913924579682539e-9,'JAK1','RPS27A','TP53'
1,6.91394718321905e-9,'MET','RPS27A','TP53'
2,6.915061333081666e-9,'RUNX3','RPS27A','TP53'
3,6.915511524940828e-9,'FANCM','RPS27A','TP53'
4,6.915904177424117e-9,'ZNRF3','RPS27A','TP53'
...,...,...,...,...
2988802,0.0008377924550676475,'KLK14','SPINK6','KLK12'
2988803,0.0008377924550676475,'KLK5','SPINK6','KLK12'
2988804,0.0010785044231135148,'BSG','SPN','SIGLEC1'
2988805,0.0010785044231135148,'SELE','SPN','SIGLEC1'


In [7]:
df_non_aggr = ParseDataframe(df_non_aggr)
df_non_aggr

/tmp/ipykernel_10329/683794502.py:6: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df[['path_weight','brol']] = df['path_weight'].str.rsplit('-',1,expand=True)


,path_weight,gene1,gene2,gene3
0,2.3694198740483993e-9,'SYNJ1','RPS27A','DVL2'
1,2.4614985340853537e-9,'TRIM69','RPS27A','PSMC5'
2,2.4855386267326423e-9,'LYN','RPS27A','PSMC5'
3,2.5069075017698555e-9,'TRIM69','RPS27A','DVL2'
4,2.5081800171847185e-9,'WNK1','RPS27A','PSMC5'
...,...,...,...,...
9529827,0.00022555979401308617,'EP300','ARID5B','PHF2'
9529828,0.00023904597429109115,'EP300','OPA1','OMA1'
9529829,0.00027713655631736125,'ATP4A','ATP4B','ATP12A'
9529830,0.0003342975443029915,'APOB','STAB1','SPARC'


In [8]:
df_ranking_aggr = pd.read_csv(ranking_file_aggr,sep='[',skiprows=0,header=None)
df_ranking_non_aggr = pd.read_csv(ranking_file_non_aggr,sep='[',skiprows=0,header=None)

In [30]:
ranking_genes_aggr = df_ranking_aggr[0].values.tolist()
ranking_genes_non_aggr = df_ranking_non_aggr[0].values.tolist()
df_ranking_top_aggr = df_ranking_aggr[0:N]
df_ranking_top_non_aggr = df_ranking_non_aggr[0:N]
top_genes_aggr = df_ranking_top_aggr[0].values.tolist()
top_genes_non_aggr = df_ranking_top_non_aggr[0].values.tolist()
top_genes2_aggr = []
top_genes2_non_aggr = []
for i in top_genes_aggr:
    a = '\''+i+'\''
    top_genes2_aggr.append(a)
for i in top_genes_non_aggr:
    a = '\''+i+'\''
    top_genes2_non_aggr.append(a)

In [10]:
top_genes_together = []
for i in top_genes_aggr:
    top_genes_together.append(i)
for i in top_genes_non_aggr:
    if i not in top_genes_together:
        top_genes_together.append(i)
        
top_genes2_together = []
for i in top_genes2_aggr:
    top_genes2_together.append(i)
for i in top_genes2_non_aggr:
    if i not in top_genes2_together:
        top_genes2_together.append(i)

In [11]:
print(len(top_genes_aggr))
print(len(top_genes_non_aggr))
print(len(top_genes_together))
top_genes_together

35
35
53


['EP300',
 'TP53',
 'GNB1',
 'PRKACA',
 'ARRB2',
 'PRKACB',
 'PRKACG',
 'RHOA',
 'ARRB1',
 'GNG7',
 'GRK2',
 'GNG13',
 'GRK3',
 'GNAL',
 'SP1',
 'CTCF',
 'AKT1',
 'ACTG1',
 'KRAS',
 'E2F1',
 'PGR',
 'POLR2B',
 'ESR1',
 'PTPRD',
 'NOTCH1',
 'FOXA1',
 'NGF',
 'RAC3',
 'PSMC1',
 'TDRKH',
 'HSPA8',
 'MYH6',
 'SP3',
 'MYH8',
 'PCDHB8',
 'STAT1',
 'STAT2',
 'RAC1',
 'CTNNB1',
 'MYC',
 'RPS27A',
 'TCF12',
 'STAT3',
 'EGR1',
 'GPLD1',
 'CTBP2',
 'ACTB',
 'JUN',
 'FOS',
 'REST',
 'HRAS',
 'NFKB1',
 'RAD21']

In [12]:
df_final_aggr = df_aggr[df_aggr['gene1'].isin(top_genes2_together) | df_aggr['gene2'].isin(top_genes2_together) | df_aggr['gene3'].isin(top_genes2_together)]
df_final_aggr

,path_weight,gene1,gene2,gene3
0,6.913924579682539e-9,'JAK1','RPS27A','TP53'
1,6.91394718321905e-9,'MET','RPS27A','TP53'
2,6.915061333081666e-9,'RUNX3','RPS27A','TP53'
3,6.915511524940828e-9,'FANCM','RPS27A','TP53'
4,6.915904177424117e-9,'ZNRF3','RPS27A','TP53'
...,...,...,...,...
2988738,0.00018032405748875292,'SLITRK5','PTPRD','SLITRK3'
2988770,0.00031190051196326556,'EGR1','CD9','IZUMO1'
2988784,0.0003397662090684561,'SP1','CD9','IZUMO1'
2988796,0.0004980731147173406,'TDRKH','PIWIL1','TDRD6'


In [13]:
df_final_non_aggr = df_non_aggr[df_non_aggr['gene1'].isin(top_genes2_together) | df_non_aggr['gene2'].isin(top_genes2_together) | df_non_aggr['gene3'].isin(top_genes2_together)]
df_final_non_aggr

,path_weight,gene1,gene2,gene3
0,2.3694198740483993e-9,'SYNJ1','RPS27A','DVL2'
1,2.4614985340853537e-9,'TRIM69','RPS27A','PSMC5'
2,2.4855386267326423e-9,'LYN','RPS27A','PSMC5'
3,2.5069075017698555e-9,'TRIM69','RPS27A','DVL2'
4,2.5081800171847185e-9,'WNK1','RPS27A','PSMC5'
...,...,...,...,...
9529821,0.00018450322083596394,'SP1','SLC5A1','RSC1A1'
9529822,0.0001856982616107296,'CTCF','TNFRSF8','TNFSF8'
9529827,0.00022555979401308617,'EP300','ARID5B','PHF2'
9529828,0.00023904597429109115,'EP300','OPA1','OMA1'


In [14]:
def calculateEdges(df,df_final,top_genes,top_genes2):
    edges = {}
    Normalization = sum(df['path_weight'].astype(float)) # Explained in the paper of OMEN
    for i in range(N):
        for j in range(N-1-i):
            node1 = top_genes[i]
            node2 = top_genes[i+1+j]
            nodes = []
            nodes.append(top_genes2[i])
            nodes.append(top_genes2[i+1+j])
            df_edge = df_final[ (df_final['gene1'].isin(nodes) & df_final['gene2'].isin(nodes)) | (df_final['gene1'].isin(nodes) & df_final['gene3'].isin(nodes)) | (df_final['gene2'].isin(nodes) & df_final['gene3'].isin(nodes))]
            s = sum(df_edge['path_weight'].astype(float))/Normalization
            if s>0:
                edges[(node1,node2)] = s*100000 # to have bigger values
    return edges 

In [15]:
edges_aggr = calculateEdges(df_aggr,df_final_aggr,top_genes_together,top_genes2_together)

In [16]:
edges_non_aggr = calculateEdges(df_non_aggr,df_final_non_aggr,top_genes_together,top_genes2_together)

In [17]:
edges_aggr

{('EP300', 'TP53'): 8.080016928174267,
 ('EP300', 'PRKACA'): 2.987071258782131,
 ('EP300', 'PRKACB'): 3.241855746068899,
 ('EP300', 'PRKACG'): 3.2522912518067963,
 ('EP300', 'RHOA'): 0.7570945899030257,
 ('EP300', 'SP1'): 6.764522056207862,
 ('EP300', 'CTCF'): 5.620066118218704,
 ('EP300', 'AKT1'): 4.711037706261807,
 ('EP300', 'ACTG1'): 0.2713322582265983,
 ('EP300', 'KRAS'): 4.165424286245783,
 ('EP300', 'E2F1'): 7.519858314245866,
 ('EP300', 'PGR'): 10.345210192623375,
 ('EP300', 'POLR2B'): 4.508459781324961,
 ('EP300', 'ESR1'): 3.0891248960162794,
 ('EP300', 'NOTCH1'): 8.138087443979655,
 ('EP300', 'FOXA1'): 6.319457265521736,
 ('EP300', 'NGF'): 0.5457278001892416,
 ('EP300', 'RAC3'): 0.3614868159090123,
 ('EP300', 'PSMC1'): 0.37657029566995587,
 ('EP300', 'HSPA8'): 3.836450436954634,
 ('EP300', 'MYH6'): 0.7569102485032604,
 ('EP300', 'SP3'): 1.9100728927912536,
 ('EP300', 'MYH8'): 0.4908846308509569,
 ('EP300', 'PCDHB8'): 0.05351901311714622,
 ('TP53', 'GNB1'): 0.2267693711727668,

In [18]:
edges_non_aggr

{('EP300', 'TP53'): 3.371480322008594,
 ('EP300', 'PRKACA'): 2.5107671999968892,
 ('EP300', 'PRKACB'): 2.0169662384171168,
 ('EP300', 'PRKACG'): 2.0233474751215965,
 ('EP300', 'SP1'): 2.2022518480077524,
 ('EP300', 'CTCF'): 2.8918570424359187,
 ('EP300', 'AKT1'): 2.0169310969740377,
 ('EP300', 'KRAS'): 1.8668891564551877,
 ('EP300', 'E2F1'): 2.4905187395574915,
 ('EP300', 'PGR'): 1.4175472766352621,
 ('EP300', 'POLR2B'): 1.4922698383433175,
 ('EP300', 'ESR1'): 2.3181944988861494,
 ('EP300', 'NOTCH1'): 1.6886041718197757,
 ('EP300', 'FOXA1'): 3.3037781745026975,
 ('EP300', 'NGF'): 0.07755575764568763,
 ('EP300', 'HSPA8'): 1.9155240492834986,
 ('EP300', 'MYH6'): 0.130700297690683,
 ('EP300', 'SP3'): 0.2842228043726027,
 ('EP300', 'MYH8'): 0.08523189886332098,
 ('TP53', 'SP1'): 3.142011110478851,
 ('TP53', 'AKT1'): 2.3269344281037747,
 ('TP53', 'E2F1'): 3.489930410085945,
 ('TP53', 'FOXA1'): 3.7253985539751557,
 ('TP53', 'NGF'): 0.05928576694808375,
 ('TP53', 'PSMC1'): 1.831105794616542,


In [19]:
print(len(edges_aggr))
print(len(edges_non_aggr))

406
187


In [20]:
edges_difference_weight = {} # weight = weight aggressive - weight non-aggressive
for i in edges_aggr:
    if i in edges_non_aggr:
        edges_difference_weight[i] = edges_aggr[i]-edges_non_aggr[i]
    else:
        edges_difference_weight[i] = edges_aggr[i]
for i in edges_non_aggr:
    if i not in edges_difference_weight:
        edges_difference_weight[i] = -edges_non_aggr[i]
print(len(edges_difference_weight))
edges_difference_weight

407


{('EP300', 'TP53'): 4.708536606165673,
 ('EP300', 'PRKACA'): 0.47630405878524185,
 ('EP300', 'PRKACB'): 1.224889507651782,
 ('EP300', 'PRKACG'): 1.2289437766851998,
 ('EP300', 'RHOA'): 0.7570945899030257,
 ('EP300', 'SP1'): 4.562270208200109,
 ('EP300', 'CTCF'): 2.728209075782785,
 ('EP300', 'AKT1'): 2.6941066092877697,
 ('EP300', 'ACTG1'): 0.2713322582265983,
 ('EP300', 'KRAS'): 2.2985351297905954,
 ('EP300', 'E2F1'): 5.0293395746883744,
 ('EP300', 'PGR'): 8.927662915988112,
 ('EP300', 'POLR2B'): 3.0161899429816437,
 ('EP300', 'ESR1'): 0.77093039713013,
 ('EP300', 'NOTCH1'): 6.449483272159879,
 ('EP300', 'FOXA1'): 3.0156790910190385,
 ('EP300', 'NGF'): 0.468172042543554,
 ('EP300', 'RAC3'): 0.3614868159090123,
 ('EP300', 'PSMC1'): 0.37657029566995587,
 ('EP300', 'HSPA8'): 1.9209263876711353,
 ('EP300', 'MYH6'): 0.6262099508125774,
 ('EP300', 'SP3'): 1.6258500884186509,
 ('EP300', 'MYH8'): 0.4056527319876359,
 ('EP300', 'PCDHB8'): 0.05351901311714622,
 ('TP53', 'GNB1'): 0.2267693711727

In [21]:
df_edges_difference_weight = pd.DataFrame.from_dict(edges_difference_weight,orient='index',columns=['Difference'])
df_edges_difference_weight = df_edges_difference_weight.sort_values(by=['Difference'],ascending=False)
df_edges_difference_weight

,Difference
"(MYH6, MYH8)",28.012751
"(SP1, SP3)",22.649197
"(ACTG1, MYH8)",10.839690
"(PGR, ESR1)",10.732673
"(PRKACG, PGR)",10.218385
...,...
"(GNAL, POLR2B)",0.002395
"(GRK2, E2F1)",0.002146
"(GRK3, E2F1)",0.002146
"(PRKACG, CTCF)",0.000881


In [32]:
with open(output_file,'w') as f:
    f.write('graph = {')
    f.write('\n')
    f.write('    nodes: [')
    f.write('\n')
    for i in range(len(top_genes_together)):
        f.write('        {') 
        f.write('\n')
        f.write('            id: ' + top_genes2_together[i] + ',') 
        f.write('\n')
        f.write('            rank_aggr: ' + str(1+ranking_genes_aggr.index(top_genes_together[i])) + ',') 
        f.write('\n')
        f.write('            rank_non_aggr: ' + str(1+ranking_genes_non_aggr.index(top_genes_together[i])) + ',') 
        f.write('\n')
        f.write('        },') 
        f.write('\n')
    f.write('    ],')
    f.write('\n')
    f.write('    links: [')
    f.write('\n')
    for edge in edges_difference_weight:
        f.write('        {source: '+ edge[0] +', target: '+edge[1]+', type: "pp", direction: "directed", max_cost: '+ str(edges_difference_weight[edge]) +', evidence: ""},')
        f.write('\n')
    f.write('    ],')
    f.write('\n')
    f.write('}')
f.close()